In [52]:
import numpy as np
import pandas as pd
from statsmodels.api import WLS, OLS
from sklearn.linear_model import LogisticRegression

In [58]:
N = 10000
a = np.random.uniform(0.1, 1., size=N)
b = np.random.uniform(0.1, 1., size=N)

v0 = np.random.normal(0., 1., size=N)
v1 = np.random.normal(0., 1., size=N)

p_d = 1. / (1. + np.exp(-(a + b)))
d = np.random.binomial(1., p=p_d)

y0 = 100 + 3*a + 2*b + v0
y1 = 102 + 6*a + 4*b + v1
y = (d==1)*y1 + (d==0)*y0

X = pd.DataFrame({'$D$': d, '$A$': a, '$B$': b, '$Y$': y })

In [59]:
X.head()

,$A$,$B$,$D$,$Y$
0,0.804654,0.244026,1,107.028005
1,0.485549,0.847228,0,101.537851
2,0.537685,0.836464,1,110.016315
3,0.639418,0.305663,1,106.466189
4,0.275248,0.165690,1,104.492944


In [4]:
X[X['$D$'] == 1].mean()['$Y$'] - X[X['$D$'] == 0].mean()['$Y$']

5.1812111079441365

In [5]:
(y1 - y0).mean()

4.734849005742652

In [61]:
# baseline bias?
X = pd.DataFrame({'$D$': d, '$A$': a, '$B$': b, '$Y$': y, '$Y_0$': y0, '$Y_1$': y1 })
X.groupby('$D$').mean()['$Y_0$']
# looks like it!

$D$
0    102.490610
1    102.826533
Name: $Y_0$, dtype: float64

In [62]:
# differential treatment effect bias?
X['$\delta$'] = X['$Y_1$'] - X['$Y_0$'] 
X.groupby('$D$').mean()['$\delta$']
# looks like it!

$D$
0    4.527131
1    4.838852
Name: $\delta$, dtype: float64

## (1) Identify a set Z satisfying the back door criterion

Looking at the data generating process, $D$ and $Y$ are both caused by $A$ and $B$. We need to control for both $A$ and $B$ to satisfy the back door criterion!

## (2) Compute propensity scores, $P(D=1|Z)$

In [69]:
propensity_model = LogisticRegression(C=1e12) # we don't want bias due to regularization!! use a large C.
propensity_model = propensity_model.fit(X[['$A$', '$B$']], X['$D$'])
X['$P(D=1|A,B)$'] = propensity_model.predict_proba(X[['$A$', '$B$']])[:,1]

In [71]:
X.head()

,$A$,$B$,$D$,$Y$,$Y_0$,$Y_1$,$\delta$,"$P(D=1|A,B)$"
0,0.804654,0.244026,1,107.028005,101.818088,107.028005,5.209917,0.751613
1,0.485549,0.847228,0,101.537851,101.537851,109.304363,7.766511,0.789054
2,0.537685,0.836464,1,110.016315,103.192129,110.016315,6.824186,0.796248
3,0.639418,0.305663,1,106.466189,101.839289,106.466189,4.626899,0.729696
4,0.275248,0.165690,1,104.492944,102.577566,104.492944,1.915378,0.621314


## (3) Compute weights, $w_{i, ATE} = \frac{1}{p_i}$ if $d=1$, and  $w_{i, ATE} = \frac{1}{1 - p_i}$ if $d=0$, 

In [72]:
X['$W_{ATE}$'] = (X['$D$'] == 1)* 1. / X['$P(D=1|A,B)$'] + (X['$D$'] == 0)* 1. /(1. - X['$P(D=1|A,B)$'])

In [76]:
X.head(n=5)

,$A$,$B$,$D$,$Y$,$Y_0$,$Y_1$,$\delta$,"$P(D=1|A,B)$",$W_{ATE}$
0,0.804654,0.244026,1,107.028005,101.818088,107.028005,5.209917,0.751613,1.330472
1,0.485549,0.847228,0,101.537851,101.537851,109.304363,7.766511,0.789054,4.740547
2,0.537685,0.836464,1,110.016315,103.192129,110.016315,6.824186,0.796248,1.255890
3,0.639418,0.305663,1,106.466189,101.839289,106.466189,4.626899,0.729696,1.370434
4,0.275248,0.165690,1,104.492944,102.577566,104.492944,1.915378,0.621314,1.609492


## (4) Compute the weighted regression estimate for the causal effect, with the specification $Y \sim \delta D + \alpha + \epsilon$

In [77]:
X['intercept'] = 1.
model = WLS(X['$Y$'], X[['$D$', 'intercept']], weights=X['$W_{ATE}$'])
result = model.fit(cov_type='HC3') # use heteroskedasticity-consistent error bars
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    $Y$   R-squared:                       0.642
Model:                            WLS   Adj. R-squared:                  0.642
Method:                 Least Squares   F-statistic:                 1.584e+04
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        19:32:55   Log-Likelihood:                -20675.
No. Observations:               10000   AIC:                         4.135e+04
Df Residuals:                    9998   BIC:                         4.137e+04
Df Model:                           1                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
$D$            4.7723      0.038    125.868      0.000       4.698       4.847
intercept    102.7331      0.029   3578.570      0.000     102.677     102.789
==============================================================================
Omnibus:                       22.284   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               18.350
Skew:                           0.025   Prob(JB):                     0.000104
Kurtosis:                       2.796   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

## Compare with an OLS regression that controls for $A$ and $B$

In [78]:
X['intercept'] = 1.
model = OLS(X['$Y$'], X[['$D$', '$A$', '$B$', 'intercept']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    $Y$   R-squared:                       0.869
Model:                            OLS   Adj. R-squared:                  0.869
Method:                 Least Squares   F-statistic:                 2.206e+04
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        19:33:45   Log-Likelihood:                -14923.
No. Observations:               10000   AIC:                         2.985e+04
Df Residuals:                    9996   BIC:                         2.988e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
$D$            4.6155      0.025    183.985      0.000       4.566       4.665
$A$            5.2805      0.042    126.972      0.000       5.199       5.362
$B$            3.4478      0.042     82.881      0.000       3.366       3.529
intercept     98.1063      0.037   2668.868      0.000      98.034      98.178
==============================================================================
Omnibus:                        1.636   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.441   Jarque-Bera (JB):                1.630
Skew:                          -0.009   Prob(JB):                        0.443
Kurtosis:                       3.060   Cond. No.                         7.41
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Looks good! the interval for the $D$ coefficient contains the value estimated above for `(y1 - y0).mean()`. What happens if the propensity model was mis-specified?

In [80]:
N = 1000000
a = np.random.uniform(0.1, 1., size=N)
b = np.random.uniform(0.1, 1., size=N)

v0 = np.random.normal(0., 1., size=N)
v1 = np.random.normal(0., 1., size=N)

p_d = 1. / (1. + np.exp(-(a + b + a*b)))  # now, the propensity model has the extra non-linearity from a*b!
d = np.random.binomial(1., p=p_d)

y1 = 102 + 3*a + 2*b + 6*a*b + v1
y0 = 100 + 2*a + 1*b - 2*a*b + v0
y = (d==1)*y1 + (d==0)*y0

X = pd.DataFrame({'$D$': d, '$A$': a, '$B$': b, '$Y$': y })



In [81]:
X[X['$D$'] == 1].mean()['$Y$'] - X[X['$D$'] == 0].mean()['$Y$']

5.813625741696569

In [82]:
(y1 - y0).mean()

5.521747543159587

In [83]:
propensity_model = LogisticRegression(C=1e10) # we don't want bias due to regularization!! use a large C.
propensity_model = propensity_model.fit(X[['$A$', '$B$']], X['$D$'])
X['$P(D=1|A,B)$'] = propensity_model.predict_proba(X[['$A$', '$B$']])[:,1]

In [84]:
X['$W_{ATE}$'] = (X['$D$'] == 1)* 1. / X['$P(D=1|A,B)$'] + (X['$D$'] == 0)* 1. /(1. - X['$P(D=1|A,B)$'])

In [85]:
X['intercept'] = 1.
model = WLS(X['$Y$'], X[['$D$', 'intercept']], weights=X['$W_{ATE}$'])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    $Y$   R-squared:                       0.689
Model:                            WLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                 2.212e+06
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        19:36:27   Log-Likelihood:            -2.1349e+06
No. Observations:             1000000   AIC:                         4.270e+06
Df Residuals:                  999998   BIC:                         4.270e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
$D$            5.5168      0.004   1487.271      0.000       5.510       5.524
intercept    101.0504      0.003   3.85e+04      0.000     101.045     101.056
==============================================================================
Omnibus:                     6407.987   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5836.864
Skew:                           0.148   Prob(JB):                         0.00
Kurtosis:                       2.772   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Not bad! We're tolerant to a little mis-specification. That's probably partly a product of our specific data generating process. 

## Now let's try estimating the ATC and ATT

Remember, $w_{i ,ATC} = \frac{1-p_i}{p_i}$ if $d_i = 1$ and one if $d_i=0$. Also, $w_{i ,ATT} = \frac{p_i}{1-p_i}$ if $d_i = 0$ and one if $d_i=1$


In [87]:
X['$W_{ATC}$'] = (X['$D$'] == 1)* (1. - X['$P(D=1|A,B)$']) / X['$P(D=1|A,B)$']  + (X['$D$'] == 0)* 1.

In [88]:
X['$W_{ATT}$'] = (X['$D$'] == 1)* 1. + (X['$D$'] == 0)* X['$P(D=1|A,B)$'] /(1. - X['$P(D=1|A,B)$'])

In [93]:
model = WLS(X['$Y$'], X[['$D$','intercept']], weights=X['$W_{ATT}$'])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    $Y$   R-squared:                       0.704
Model:                            WLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                 2.378e+06
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        19:51:30   Log-Likelihood:            -2.1396e+06
No. Observations:             1000000   AIC:                         4.279e+06
Df Residuals:                  999998   BIC:                         4.279e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
$D$            5.7311      0.004   1542.173      0.000       5.724       5.738
intercept    101.0681      0.003   3.84e+04      0.000     101.063     101.073
==============================================================================
Omnibus:                    18378.649   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19005.140
Skew:                           0.327   Prob(JB):                         0.00
Kurtosis:                       2.828   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
model = WLS(X['$Y$'], X[['$D$','intercept']], weights=X['$W_{ATC}$'])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    $Y$   R-squared:                       0.657
Model:                            WLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                 1.913e+06
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        14:50:20   Log-Likelihood:            -2.1001e+06
No. Observations:             1000000   AIC:                         4.200e+06
Df Residuals:                  999998   BIC:                         4.200e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
$D$            4.7256      0.003   1383.086      0.000       4.719       4.732
intercept    100.9807      0.002   4.18e+04      0.000     100.976     100.985
==============================================================================
Omnibus:                    52307.546   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            60870.268
Skew:                          -0.593   Prob(JB):                         0.00
Kurtosis:                       3.231   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## What are the true values?

In [90]:
X['$Y_1$'] = y1
X['$Y_0$'] = y0

In [91]:
treated = X[X['$D$'] == 1]
ATT = (treated['$Y_1$'] - treated['$Y_0$']).mean()
ATT

5.739338141875647

In [92]:
control = X[X['$D$'] == 0]
ATC = (control['$Y_1$'] - control['$Y_0$']).mean()
ATC

4.714818294637095

## So it looks good!

In [115]:
N = 1000000
a = np.random.uniform(0.1, 1., size=N)
b = np.random.uniform(0.1, 1., size=N)

v0 = np.random.normal(0., 1., size=N)
v1 = np.random.normal(0., 1., size=N)

p_d = 1. / (1. + np.exp(-(a + b + a*b)))  # now, the propensity model has the extra non-linearity from a*b!
d = np.random.binomial(1., p=p_d)

y1 = 102 + 3*a + 2*b + 6*a*b + v1
y0 = 100 + 2*a + 1*b - 2*a*b + v0
y = (d==1)*y1 + (d==0)*y0

X = pd.DataFrame({'$D$': d, '$A$': a, '$B$': b, '$Y$': y })



In [95]:
(y1 - y0).mean()

5.492944413986289

In [116]:
X[X['$D$'] == 1].mean()['$Y$'] - X[X['$D$'] == 0].mean()['$Y$']

5.816296771833464

In [117]:
propensity_model = LogisticRegression(C=1e15) # we don't want bias due to regularization!! use a large C.
propensity_model = propensity_model.fit(X[['$A$', '$B$']], X['$D$'])
X['$P(D=1|A,B)$'] = propensity_model.predict_proba(X[['$A$', '$B$']])[:,1]

X['$W_{ATE}$'] = (X['$D$'] == 1)* 1. / X['$P(D=1|A,B)$'] + (X['$D$'] == 0)* 1. /(1. - X['$P(D=1|A,B)$'])

In [99]:
X['intercept'] = 1.
X['$A^2$'] = X['$A$'] ** 2
X['$B^2$'] = X['$B$'] ** 2
X['$AB$'] = X['$B$'] * X['$A$']

model = WLS(X['$Y$'], X[['$D$', '$A^2$', '$B^2$', '$A$', '$B$', '$AB$', 'intercept']], weights=X['$W_{ATE}$'])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    $Y$   R-squared:                       0.816
Model:                            WLS   Adj. R-squared:                  0.816
Method:                 Least Squares   F-statistic:                     7390.
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        19:55:46   Log-Likelihood:                -18758.
No. Observations:               10000   AIC:                         3.753e+04
Df Residuals:                    9993   BIC:                         3.758e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
$D$            5.4907      0.029    191.896      0.000       5.435       5.547
$A^2$          0.2355      0.239      0.987      0.324      -0.232       0.703
$B^2$         -0.0393      0.235     -0.167      0.867      -0.500       0.421
$A$            2.3316      0.293      7.956      0.000       1.757       2.906
$B$            1.6422      0.289      5.684      0.000       1.076       2.209
$AB$           2.0262      0.211      9.609      0.000       1.613       2.440
intercept     98.1921      0.112    873.772      0.000      97.972      98.412
==============================================================================
Omnibus:                     1708.714   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8378.454
Skew:                          -0.745   Prob(JB):                         0.00
Kurtosis:                       7.229   Cond. No.                         43.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## So we have a doubly robust regression by including $A$ and $B$ in the regression model, as well as the propensity score model!

## Can we use more general models with weighting and the g-formula?

## Check the naive estimate first:

In [100]:
from keras.models import Model
from keras.layers import Input, Dense

x_variables = ['$D$']

x_in = Input(shape=(len(x_variables),))
h1 = Dense(100, activation='relu')(x_in)
h2 = Dense(100, activation='relu')(h1)
y_out = Dense(1, activation='linear')(h2)

model = Model(inputs=[x_in], outputs=[y_out])
model.compile(loss='mean_squared_error', optimizer='RMSprop')

model.fit(X[x_variables], X['$Y$'], epochs=10)

Epoch 1/10
10000/10000 [==============================] - 1s 125us/step - loss: 4501.6635
Epoch 2/10
10000/10000 [==============================] - 1s 92us/step - loss: 104.1061
Epoch 3/10
10000/10000 [==============================] - 1s 95us/step - loss: 5.2603
Epoch 4/10
10000/10000 [==============================] - 1s 87us/step - loss: 5.2251
Epoch 5/10
10000/10000 [==============================] - 1s 94us/step - loss: 5.2519
Epoch 6/10
10000/10000 [==============================] - 1s 94us/step - loss: 5.2839
Epoch 7/10
10000/10000 [==============================] - 1s 98us/step - loss: 5.2317
Epoch 8/10
10000/10000 [==============================] - 1s 93us/step - loss: 5.2134
Epoch 9/10
10000/10000 [==============================] - 1s 98us/step - loss: 5.2928
Epoch 10/10
10000/10000 [==============================] - 1s 95us/step - loss: 5.2210


In [101]:
df = X.copy()
df['$D$'] = 1
y1_hat = model.predict(df['$D$'])
df['$D$'] = 0
y0_hat = model.predict(df['$D$'])

In [102]:
(y1_hat - y0_hat).mean()

5.8828003607486465

In [39]:
## close to the naive estimator! Now, let's try weighting...

In [106]:
from keras.models import Model
from keras.layers import Input, Dense

x_variables = ['$D$']

x_in = Input(shape=(len(x_variables),))
h1 = Dense(100, activation='relu')(x_in)
h2 = Dense(100, activation='relu')(h1)
y_out = Dense(1, activation='linear')(h2)

model = Model(inputs=[x_in], outputs=[y_out])
model.compile(loss='mean_squared_error', optimizer='RMSprop')

model.fit(X[x_variables], X['$Y$'], sample_weight=X['$W_{ATE}$'], epochs=10)

Epoch 1/10
10000/10000 [==============================] - 2s 155us/step - loss: 9988.2020
Epoch 2/10
10000/10000 [==============================] - 1s 122us/step - loss: 382.4181
Epoch 3/10
10000/10000 [==============================] - 1s 120us/step - loss: 7.5005
Epoch 4/10
10000/10000 [==============================] - 1s 118us/step - loss: 7.4625
Epoch 5/10
10000/10000 [==============================] - 1s 114us/step - loss: 7.4858
Epoch 6/10
10000/10000 [==============================] - 1s 116us/step - loss: 7.4664
Epoch 7/10
10000/10000 [==============================] - 1s 114us/step - loss: 7.4838
Epoch 8/10
10000/10000 [==============================] - 1s 117us/step - loss: 7.4989
Epoch 9/10
10000/10000 [==============================] - 1s 119us/step - loss: 7.4709
Epoch 10/10
10000/10000 [==============================] - 1s 125us/step - loss: 7.4444


In [107]:
df = X.copy()
df['$D$'] = 1
y1_hat = model.predict(df['$D$'])
df['$D$'] = 0
y0_hat = model.predict(df['$D$'])

In [108]:
(y1_hat - y0_hat).mean()

5.9776258569544325

## Much better! Now, let's try doubly-robust ...

In [119]:
from keras.models import Model
from keras.layers import Input, Dense

x_variables = ['$D$', '$A$', '$B$']

x_in = Input(shape=(len(x_variables),))
h1 = Dense(100, activation='relu')(x_in)
h2 = Dense(100, activation='relu')(h1)
y_out = Dense(1, activation='linear')(h2)

model = Model(inputs=[x_in], outputs=[y_out])
model.compile(loss='mean_squared_error', optimizer='RMSprop')

model.fit(X[x_variables], X['$Y$'], sample_weight=X['$W_{ATE}$'], epochs=1)

Epoch 1/1
1000000/1000000 [==============================] - 119s 119us/step - loss: 64.2769


In [120]:
df = X.copy()
df['$D$'] = 1
y1_hat = model.predict(df[x_variables])
df['$D$'] = 0
y0_hat = model.predict(df[x_variables])

In [121]:
(y1_hat - y0_hat).mean()

6.051592889997136

## Let's go crazy -- feedforward nets for everything!

In [47]:
from keras.models import Model
from keras.layers import Input, Dense

x_variables = ['$A$', '$B$']

x_in = Input(shape=(len(x_variables),))
h1 = Dense(100, activation='relu')(x_in)
h2 = Dense(100, activation='relu')(h1)
y_out = Dense(1, activation='linear')(h2)

propensity_model = Model(inputs=[x_in], outputs=[y_out])
propensity_model.compile(loss='mean_squared_error', optimizer='RMSprop')

propensity_model.fit(X[x_variables], X['$D$'], epochs=10)

X['$P(D=1|A,B)$'] = propensity_model.predict(X[['$A$', '$B$']])

X['$W_{ATE}$'] = (X['$D$'] == 1)* 1. / X['$P(D=1|A,B)$'] + (X['$D$'] == 0)* 1. /(1. - X['$P(D=1|A,B)$'])

Epoch 1/10
10000/10000 [==============================] - 1s 127us/step - loss: 0.1721
Epoch 2/10
10000/10000 [==============================] - 1s 86us/step - loss: 0.1607
Epoch 3/10
10000/10000 [==============================] - 1s 90us/step - loss: 0.1603
Epoch 4/10
10000/10000 [==============================] - 1s 85us/step - loss: 0.1602
Epoch 5/10
10000/10000 [==============================] - 1s 86us/step - loss: 0.1600
Epoch 6/10
10000/10000 [==============================] - 1s 86us/step - loss: 0.1600
Epoch 7/10
10000/10000 [==============================] - 1s 85us/step - loss: 0.1598
Epoch 8/10
10000/10000 [==============================] - 1s 86us/step - loss: 0.1596
Epoch 9/10
10000/10000 [==============================] - 1s 88us/step - loss: 0.1600
Epoch 10/10
10000/10000 [==============================] - 1s 90us/step - loss: 0.1601


In [48]:
from keras.models import Model
from keras.layers import Input, Dense

x_variables = ['$D$']

x_in = Input(shape=(len(x_variables),))
h1 = Dense(100, activation='relu')(x_in)
h2 = Dense(100, activation='relu')(h1)
y_out = Dense(1, activation='linear')(h2)

model = Model(inputs=[x_in], outputs=[y_out])
model.compile(loss='mean_squared_error', optimizer='RMSprop')

model.fit(X[x_variables], X['$Y$'], sample_weight=X['$W_{ATE}$'], epochs=10)

Epoch 1/10
10000/10000 [==============================] - 2s 179us/step - loss: 9558.9367
Epoch 2/10
10000/10000 [==============================] - 1s 127us/step - loss: 294.8762
Epoch 3/10
10000/10000 [==============================] - 2s 150us/step - loss: 7.6703
Epoch 4/10
10000/10000 [==============================] - 1s 114us/step - loss: 7.7014
Epoch 5/10
10000/10000 [==============================] - 1s 116us/step - loss: 7.6981
Epoch 6/10
10000/10000 [==============================] - 1s 123us/step - loss: 7.6631
Epoch 7/10
10000/10000 [==============================] - 1s 129us/step - loss: 7.6702
Epoch 8/10
10000/10000 [==============================] - 1s 132us/step - loss: 7.6746
Epoch 9/10
10000/10000 [==============================] - 1s 133us/step - loss: 7.6224
Epoch 10/10
10000/10000 [==============================] - 1s 128us/step - loss: 7.6716


In [49]:
df = X.copy()
df['$D$'] = 1
y1_hat = model.predict(df[x_variables])
df['$D$'] = 0
y0_hat = model.predict(df[x_variables])

In [50]:
(y1_hat - y0_hat).mean()

5.458013286568061

In [51]:
(y1 - y0).mean()

5.562727563487781